In [1]:
# run time record
time.start <- Sys.time()
print(paste('start time:',time.start))

[1] "start time: 2022-02-09 13:16:08"


In [2]:
library(MLmetrics)
library(Seurat)
library(SeuratDisk)
library(caret)


Attaching package: ‘MLmetrics’


The following object is masked from ‘package:base’:

    Recall


Attaching SeuratObject

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ggplot2

Loading required package: lattice


Attaching package: ‘caret’


The following objects are masked from ‘package:MLmetrics’:

    MAE, RMSE




In [3]:
##h5ad data -> h5seurat data
# library(SeuratDisk)
# Convert ( "../scClass_data/cordblood_pp_match.h5ad" , dest  =  "h5seurat" , overwrite  =  FALSE )
# Convert ("../scClass_data/pbmc68k_v2.h5ad", dest="h5seurat", overwrite=TRUE) #if the file exist, then overwrite

In [4]:
ref.file <- "../scClass_data/pbmcmm_pp_subsample.h5seurat"
test.file <- "../scClass_data/cordblood_pp_match.h5seurat"

In [5]:
ref <- LoadH5Seurat(ref.file)

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [6]:
ref <- NormalizeData(ref, verbose = FALSE)
ref <- FindVariableFeatures(ref, selection.method = "vst", nfeatures = 2000,verbose = FALSE)

In [7]:
ref <- ScaleData(ref, verbose = FALSE)
ref <- RunPCA(ref, npcs = 30, verbose = FALSE)
ref <- RunUMAP(ref, reduction = "pca", dims = 1:30, verbose = FALSE)

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”


## Cell type classification

In [8]:
test <- LoadH5Seurat(test.file)

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [9]:
ref.anchors <- FindTransferAnchors(reference = ref, query =test, dims = 1:30, reference.reduction = "pca")

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 6094 anchors

Filtering anchors

	Retained 517 anchors



In [10]:
ref$transfer_cell_type <- as.character(ref$transfer_id) # celltype = transfer_id (int to char)
predictions <- TransferData(anchorset = ref.anchors, refdata = ref$transfer_cell_type,dims = 1:30)
test <- AddMetaData(test, metadata = predictions)

Finding integration vectors

Finding integration vector weights

Predicting cell labels



## Make Confusion Matrix and evaluate accuracy

In [11]:
label <- c('helper T cell','cytotoxic T cell','memory B cell','naive B cell',
           'plasma cell','natural killer cell','erythrocyte',
           'megakaryocyte','monocyte','dendritic cell','HSPC')

In [12]:
y.pred <- factor(label[as.integer(test$predicted.id)+1],levels=label)
y.true <- factor(test$transfer_cell_type,levels=label)

In [13]:
y.pred <- y.pred[!is.na(y.true)]
y.true <- y.true[!is.na(y.true)]

In [14]:
mat.table <- confusionMatrix(y.true,y.pred)
mat.df <- as.data.frame.matrix(mat.table$table) 
colnames(mat.df) = 0:10
mat.df

,0,1,2,3,4,5,6,7,8,9,10
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
helper T cell,0,26165,0,3021,0,54,1080,0,18,0,10
cytotoxic T cell,0,12467,0,333,0,6,192,0,0,0,4
memory B cell,0,26,0,708,0,2,2,0,0,2,0
naive B cell,0,0,0,14488,0,0,10,0,0,0,0
plasma cell,0,0,0,0,0,0,0,0,0,0,0
natural killer cell,0,980,0,439,0,3923,258,0,3,0,3
erythrocyte,0,22,0,99,0,7,1177,11,1,4,928
megakaryocyte,0,411,0,107,0,8,13,304,10,3,40
monocyte,0,79,0,83,0,0,130,4,5283,3347,1


In [15]:
mat.df.percent <- round(mat.df/(rowSums(mat.df)+0.001)*100, digits=1)
mat.df.percent
print(paste('acc:',sum(y.true==y.pred)/length(y.true)*100,'%'))

,0,1,2,3,4,5,6,7,8,9,10
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
helper T cell,0,86.2,0,10.0,0,0.2,3.6,0.0,0.1,0.0,0.0
cytotoxic T cell,0,95.9,0,2.6,0,0.0,1.5,0.0,0.0,0.0,0.0
memory B cell,0,3.5,0,95.7,0,0.3,0.3,0.0,0.0,0.3,0.0
naive B cell,0,0.0,0,99.9,0,0.0,0.1,0.0,0.0,0.0,0.0
plasma cell,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
natural killer cell,0,17.5,0,7.8,0,70.0,4.6,0.0,0.1,0.0,0.1
erythrocyte,0,1.0,0,4.4,0,0.3,52.3,0.5,0.0,0.2,41.3
megakaryocyte,0,45.9,0,11.9,0,0.9,1.5,33.9,1.1,0.3,4.5
monocyte,0,0.9,0,0.9,0,0.0,1.5,0.0,59.2,37.5,0.0


[1] "acc: 49.7209828687384 %"


In [16]:
evaluation_metrics<-function(performance){
    eva.list <- c()
    performance.name <- as.character(substitute(performance))
    for (i in label)
    {
        eva <- performance(y.true,y.pred, positive = i)
        if (!is.nan(eva))
        {
            eva.list <- c(eva.list,eva)
        }
    }
    print(paste(c(paste(performance.name,':'),eva.list)))
    print('-----------------------------------------------------------------')
    print(paste(performance.name,'(average):',mean(eva.list)))
}

In [17]:
evaluation_metrics(Precision)

[1] "Precision :"       "0.310502851734701" "0.75047915047915" 
[4] "0.98075"           "0.411250873515024" "0.952978056426332"
[7] "0.993792325056433" "0.147790756729304" "0"                
[1] "-----------------------------------------------------------------"
[1] "Precision (average): 0.568443001742618"


In [18]:
evaluation_metrics(Recall)

 [1] "Recall :"          "0"                 "0.958852484233195"
 [4] "0"                 "0.999310249689612" "0.699785943631823"
 [7] "0.523343708314807" "0.339285714285714" "0.591800156827602"
[10] "0.95253682487725" 
[1] "-----------------------------------------------------------------"
[1] "Recall (average): 0.562768342428889"


In [19]:
evaluation_metrics(F1_Score)

[1] "F1_Score :"        "0.469098639775742" "0.857202023489039"
[4] "0.816781178430148" "0.460575229896302" "0.500411522633745"
[7] "0.741838095906761" "0.255880413277643"
[1] "-----------------------------------------------------------------"
[1] "F1_Score (average): 0.58596958620134"


In [20]:
# run time record
time.end <- Sys.time()
print(paste('start time:',time.start))
print(paste('end   time:',time.end))
time.end - time.start 

[1] "start time: 2022-02-09 13:16:08"
[1] "end   time: 2022-02-09 13:22:58"


Time difference of 6.829717 mins